In [15]:
r = range(1000)      # sequnce
g = (x for x in r)   # generator
l = list(r)          # sequence
i = iter(l)          # iterator

Compare the size of sequence, generator and iterator

In [16]:
import sys
print('range:', sys.getsizeof(r))
print('iter :', sys.getsizeof(i))
print('gen  :', sys.getsizeof(g))
print('seq  :', sys.getsizeof(l))

range: 48
iter : 48
gen  : 112
seq  : 8056


Compare the attributes/methods of sequence, generator and iterator

In [17]:
def attrs(obj):
    return ', '.join(a for a in ['__iter__', '__len__', 'send'] if hasattr(obj, a))

In [18]:
print('range:', attrs(r))
print('iter :', attrs(i))
print('gen  :', attrs(g))
print('seq  :', attrs(l))

range: __iter__, __len__
iter : __iter__
gen  : __iter__, send
seq  : __iter__, __len__


In [19]:
# simple example for yield from and multiple yields
def gen():    
    yield from [1 , 2, 3]
    yield from [4, 5]

In [20]:
list(gen())

[1, 2, 3, 4, 5]

In [21]:
# a grep-like function. Here the eager implementation.
def find(query, pat='*.*'):
    from glob import glob
    hits = []    
    for filepath in glob(pat):
        with open(filepath) as lines:
            for line in lines:
                for word in line.split():
                    if query in word.lower():
                        hits.append(word)
    return hits                        
find('generator', '*.md')                        

['011-generators', 'generators']

In [22]:
def filepaths(pat='*.*'):
    from glob import glob
    return glob(pat)

def lines(filepaths):
    for filepath in filepaths:
        with open(filepath) as lines:
            yield from lines

def words(lines):
    for line in lines: 
        yield from line.split()
        
def find(query, pat='*.*'):
    ws = words(lines(filepaths(pat)))
    return (w for w in ws if query in w)

list(find('generator', '*.md'))

['011-generators', 'generators']

The same code with nutsflow

In [35]:
from nutsflow import *
from glob import glob

@nut_processor
def lines(filepaths):
    for filepath in filepaths:
        yield from open(filepath)
      
@nut_processor    
def words(lines):
    for line in lines: 
        yield from line.split()  
           
def find(query, pat='*.*'):            
    return glob('*.md') >> lines() >> words() >> Filter(lambda w: query in w) >> Collect()

find('generator', '*.md')

['011-generators', 'generators']

In [24]:
nums = [-2, -1, 0, 3, 4]

In [25]:
def absolutes(nums):
    return (abs(n) for n in nums)
absolutes(nums)

<generator object absolutes.<locals>.<genexpr> at 0x0000026D24CA1740>

In [26]:
def absolutes(nums):
    for n in nums:
        yield abs(n)
absolutes(nums)

<generator object absolutes at 0x0000026D24E554A0>

In [32]:
class absolutes:
    def __init__(self, nums):
        self.nums = iter(nums)
    def __iter__(self):
        return self
    def __next__(self):
        while not (n:=next(self.nums)):
            pass
        return abs(n)

list(absolutes([-1, -2, 0, 3, 4]))

[1, 2, 3, 4]

In [28]:
def flatten(xs):
    if isinstance(xs, list):
        for x in xs:
            yield from flatten(x)
    else:
        yield xs
        
# flatten without "yield from"        
def flatten2(xs):
    if isinstance(xs, list):
        for x in xs:
            for r in flatten(x):
                yield r
    else:
        yield xs        
        
def take(iterable, n):
    return [next(iterable) for _ in range(n)]

f = flatten([1, [2, 3, [4]], 5])
list(f)

[1, 2, 3, 4, 5]

In [29]:
# using list comprehension
def flatten(xs):
    if isinstance(xs, list):
        return [y for x in xs for y in flatten(x)]  
    return [xs]

In [30]:
flatten([1, [2, 3, [4]], 5])

[1, 2, 3, 4, 5]